In [5]:
import scipy.io as sio
import numpy as np
from os import getcwd
from os.path import join 
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.multivariate.pca import PCA
from statsmodels.tsa.ar_model import AutoReg
import pandas as pd 

In [6]:
fname = join(os.getcwd(),'data','data3SS2009.mat')

mat_contents = sio.loadmat(fname)

dataset = mat_contents['dataset']


N, Chno, Nc = dataset.shape
# N: number of samples
# Chno: number of channels
# Nc: number of cases

y = mat_contents['labels'].reshape(Nc)




In [7]:
Ch1 = dataset[:,0,:] # célula de carga: força do shaker
Ch2 = dataset[:,1,:] # acelerômetro: base
Ch3 = dataset[:,2,:] # acelerômetro: 1o andar
Ch4 = dataset[:,3,:] # acelerômetro: 2o andar
Ch5 = dataset[:,4,:] # acelerômetro: 3o andar

Ts = 3.125 * 1e-3 # sampling time
time = (np.linspace(1,N,N) - 1) * Ts

    


In [8]:
ARmodel_2 = AutoReg(Ch2[:,1],30,old_names=False).fit()
ARmodel_3 = AutoReg(Ch3[:,1],30,old_names=False).fit()
ARmodel_4 = AutoReg(Ch4[:,1],30,old_names=False).fit()
ARmodel_5 = AutoReg(Ch5[:,1],30,old_names=False).fit()
vec_2 = ARmodel_2.params
vec_3 = ARmodel_3.params
vec_4 = ARmodel_4.params
vec_5 = ARmodel_5.params
vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))

ARmodel_var = vec_temp

for i in range(1,850):
    ARmodel_2 = AutoReg(Ch2[:,i],30,old_names=False).fit()
    ARmodel_3 = AutoReg(Ch3[:,i],30,old_names=False).fit()
    ARmodel_4 = AutoReg(Ch4[:,i],30,old_names=False).fit()
    ARmodel_5 = AutoReg(Ch5[:,i],30,old_names=False).fit()
    vec_2 = ARmodel_2.params
    vec_3 = ARmodel_3.params
    vec_4 = ARmodel_4.params
    vec_5 = ARmodel_5.params
    vec_temp = np.concatenate((vec_2,vec_3,vec_4,vec_5))
    ARmodel_var = np.vstack([ARmodel_var,vec_temp])

In [9]:
X = ARmodel_var
X.shape

(850, 124)

In [10]:
import numpy as np
from scipy import stats
from scipy.stats.stats import mode
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (train_test_split, RepeatedKFold,
RandomizedSearchCV)
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn import tree
from sklearn.pipeline import Pipeline

In [11]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)
ARmodel_reduced = pca.fit(ARmodel_var).transform(ARmodel_var)

X = ARmodel_reduced

In [ ]:
pip install --upgrade tensorflow

In [22]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = ARmodel_reduced
y = mat_contents['labels'].reshape(Nc)
y = y - 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Architecture du réseau de neurone
model = tf.keras.Sequential([tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
                             tf.keras.layers.Dense(32, activation='relu'),
                             tf.keras.layers.Dense(17, activation='softmax')  ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=80, batch_size=32, validation_split=0.1)


## Test with same protocol

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Précision du test: {test_accuracy:.4f}')

predictions = model.predict(X_test)


Epoch 1/80
15/15 [==============================] - 2s 32ms/step - loss: 2.7921 - accuracy: 0.1133 - val_loss: 2.7076 - val_accuracy: 0.1961
Epoch 2/80
15/15 [==============================] - 0s 9ms/step - loss: 2.5812 - accuracy: 0.3159 - val_loss: 2.5409 - val_accuracy: 0.3333
Epoch 3/80
15/15 [==============================] - 0s 12ms/step - loss: 2.3848 - accuracy: 0.4444 - val_loss: 2.3707 - val_accuracy: 0.3922
Epoch 4/80
15/15 [==============================] - 0s 11ms/step - loss: 2.1788 - accuracy: 0.5251 - val_loss: 2.1828 - val_accuracy: 0.4902
Epoch 5/80
15/15 [==============================] - 0s 11ms/step - loss: 1.9641 - accuracy: 0.6100 - val_loss: 1.9781 - val_accuracy: 0.5490
Epoch 6/80
15/15 [==============================] - 0s 10ms/step - loss: 1.7288 - accuracy: 0.6471 - val_loss: 1.7564 - val_accuracy: 0.6275
Epoch 7/80
15/15 [==============================] - 0s 11ms/step - loss: 1.4873 - accuracy: 0.7407 - val_loss: 1.5277 - val_accuracy: 0.6667
Epoch 8/80
15/